In [1]:
!pip install netcdf4 h5netcdf xarray --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 83.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 63.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 60.8 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
import scipy
import glob
import h5py
import h5netcdf
import netCDF4 as nc
import xarray as xr
import os
import re

xr.backends.refresh_engines()
print(xr.backends.list_engines())

{'netcdf4': <NetCDF4BackendEntrypoint>
  Open netCDF (.nc, .nc4 and .cdf) and most HDF5 files using netCDF4 in Xarray
  Learn more at https://docs.xarray.dev/en/stable/generated/xarray.backends.NetCDF4BackendEntrypoint.html, 'h5netcdf': <H5netcdfBackendEntrypoint>
  Open netCDF (.nc, .nc4 and .cdf) and most HDF5 files using h5netcdf in Xarray
  Learn more at https://docs.xarray.dev/en/stable/generated/xarray.backends.H5netcdfBackendEntrypoint.html, 'scipy': <ScipyBackendEntrypoint>
  Open netCDF files (.nc, .nc4, .cdf and .gz) using scipy in Xarray
  Learn more at https://docs.xarray.dev/en/stable/generated/xarray.backends.ScipyBackendEntrypoint.html, 'store': <StoreBackendEntrypoint>
  Open AbstractDataStore instances in Xarray
  Learn more at https://docs.xarray.dev/en/stable/generated/xarray.backends.StoreBackendEntrypoint.html}


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
idw_path_year = './1 - Organized data gauge/BRAZIL/NetCDF/IDW_optimization'
idw_consolidadtion_path = './1 - Organized data gauge/BRAZIL/NetCDF/IDW_consolidation'

In [10]:
idw_path_year = "/content/drive/MyDrive/QualiBRain/Scripts e Dados/1 - Organized data gauge/BRAZIL/NetCDF/IDW_consolidation_year"
idw_path_decade = "/content/drive/MyDrive/QualiBRain/Scripts e Dados/1 - Organized data gauge/BRAZIL/NetCDF/IDW_consolidation_decade"

In [11]:
all_files = glob.glob(idw_path_year + r"/*.nc")
total_length = len(all_files)
print(total_length)
all_files[0]

64


'/content/drive/MyDrive/QualiBRain/Scripts e Dados/1 - Organized data gauge/BRAZIL/NetCDF/IDW_consolidation_year/precipitation_idw_1988.nc'

In [12]:
decade_dict = {}

# Process each file in the list
for file in all_files[:]:
    # Use regex to extract the decade from the filename; assumes pattern "yyyy-mm-dd.nc" at the end.
    match = re.search(r'(\d{4}).nc$', file)
    if match:
        decade = ( int(match.group(1))-1 ) // 10 * 10
        # Initialize list if the key does not exist
        if decade not in decade_dict:
            decade_dict[decade] = []
        # Append the file to the appropriate decade list
        decade_dict[decade].append(file)

# If you want the result as a list of lists sorted by decade:
decade_file_lists = [decade_dict[d] for d in sorted(decade_dict)]
print(decade_file_lists[0])

# Alternatively, print the dictionary with decade labels:
for decade in sorted(decade_dict):
    print(f"list_{decade}: {len(decade_dict[decade])} files; {decade_dict[decade]}")

['/content/drive/MyDrive/QualiBRain/Scripts e Dados/1 - Organized data gauge/BRAZIL/NetCDF/IDW_consolidation_year/precipitation_idw_1967.nc', '/content/drive/MyDrive/QualiBRain/Scripts e Dados/1 - Organized data gauge/BRAZIL/NetCDF/IDW_consolidation_year/precipitation_idw_1969.nc', '/content/drive/MyDrive/QualiBRain/Scripts e Dados/1 - Organized data gauge/BRAZIL/NetCDF/IDW_consolidation_year/precipitation_idw_1970.nc', '/content/drive/MyDrive/QualiBRain/Scripts e Dados/1 - Organized data gauge/BRAZIL/NetCDF/IDW_consolidation_year/precipitation_idw_1966.nc', '/content/drive/MyDrive/QualiBRain/Scripts e Dados/1 - Organized data gauge/BRAZIL/NetCDF/IDW_consolidation_year/precipitation_idw_1968.nc', '/content/drive/MyDrive/QualiBRain/Scripts e Dados/1 - Organized data gauge/BRAZIL/NetCDF/IDW_consolidation_year/precipitation_idw_1963.nc', '/content/drive/MyDrive/QualiBRain/Scripts e Dados/1 - Organized data gauge/BRAZIL/NetCDF/IDW_consolidation_year/precipitation_idw_1964.nc', '/content/dr

In [14]:
decades = []
for filename in os.listdir(idw_path_decade + r"/"):
    if filename.endswith(r".nc"):
        match = re.search(r'_(\d{4})\.nc$', filename)
        if match:
            decades.append(int(match.group(1)))
decades

[]

In [ ]:
# decades.pop(-3)

# VERIFY LAST FILE

In [15]:
# prompt: filter year_dict where key are not in years

filtered_decade_dict = {key: value for key, value in decade_dict.items() if key not in decades}


In [16]:
merged_datasets = {}

for decade, file_list in filtered_decade_dict.items():
    # Open and merge all files for this year by coordinates
    datasets = [xr.open_dataset(filepath) for filepath in file_list]
    ds_total = xr.merge(datasets)
    # del datasets

    # Save the merged dataset to a new NetCDF file (optional)
    output_filename = f'/content/drive/MyDrive/QualiBRain/Scripts e Dados/1 - Organized data gauge/BRAZIL/NetCDF/IDW_consolidation_decade/precipitation_idw_{decade}.nc'
    ds_total.to_netcdf(output_filename)
    # del ds_total
    print(f"Saved merged dataset for year {decade} to {output_filename}")

Saved merged dataset for year 1980 to /content/drive/MyDrive/QualiBRain/Scripts e Dados/1 - Organized data gauge/BRAZIL/NetCDF/IDW_consolidation_decade/precipitation_idw_1980.nc
Saved merged dataset for year 1960 to /content/drive/MyDrive/QualiBRain/Scripts e Dados/1 - Organized data gauge/BRAZIL/NetCDF/IDW_consolidation_decade/precipitation_idw_1960.nc
Saved merged dataset for year 1970 to /content/drive/MyDrive/QualiBRain/Scripts e Dados/1 - Organized data gauge/BRAZIL/NetCDF/IDW_consolidation_decade/precipitation_idw_1970.nc
Saved merged dataset for year 1990 to /content/drive/MyDrive/QualiBRain/Scripts e Dados/1 - Organized data gauge/BRAZIL/NetCDF/IDW_consolidation_decade/precipitation_idw_1990.nc
Saved merged dataset for year 2020 to /content/drive/MyDrive/QualiBRain/Scripts e Dados/1 - Organized data gauge/BRAZIL/NetCDF/IDW_consolidation_decade/precipitation_idw_2020.nc
Saved merged dataset for year 2010 to /content/drive/MyDrive/QualiBRain/Scripts e Dados/1 - Organized data gau

In [17]:
df_teste = xr.open_dataset(output_filename)
df_teste

<xarray.Dataset> Size: 734MB
Dimensions:   (lat: 158, long: 159, datetime: 3652)
Coordinates:
  * lat       (lat) float64 1kB -33.88 -33.62 -33.38 ... 4.875 5.125 5.375
  * long      (long) float64 1kB -74.12 -73.88 -73.62 ... -35.12 -34.88 -34.62
  * datetime  (datetime) datetime64[ns] 29kB 2001-01-01 ... 2010-12-31
Data variables:
    rain_mm   (lat, long, datetime) float64 734MB ...